In [1]:
import pandas as pd
import geopandas as gpd
import fiona

In [3]:
url_dict = {  
2017_2020: "https://opendata.arcgis.com/datasets/4eea8bf452e34f8c9d9ac07c54c0b4ab_0.geojson"
}

In [4]:
def merge_years(url_dict):
    df_dict={}
    for i in url_dict.keys():
        df = gpd.read_file(url_dict.get(i)) #ping url for geojson file and read in as geopandas df
        df_dict.update({i:df}) #assign each df to a year in the dict
    return(df_dict)

In [6]:
dfFood=merge_years(url_dict)

In [7]:
def merge_gpd_dfs(df_dict):
    df_list=list(df_dict.values()) #take dataframes from dict and turn into list
    df = pd.concat(df_list, ignore_index=True, sort=True) #concatenate the dataframes
    crs = df_list[0].crs #capture coordinate system from first record
    df = gpd.GeoDataFrame(df,crs=crs) # transform dataframe into geopandas dataframe
    return(df)

In [9]:
df=merge_gpd_dfs(dfFood)

In [5]:
df.head().T

,0,1,2,3,4
LIC#RB,LIC107101,LIC107101,LIC107101,LIC107101,LIC107101
LIC_Cat,RESTAURANT,RESTAURANT,RESTAURANT,RESTAURANT,RESTAURANT
Business_Name,SOTAROL UPTOWN,SOTAROL UPTOWN,SOTAROL UPTOWN,SOTAROL UPTOWN,SOTAROL UPTOWN
Risk Lvl,1,1,1,1,1
Full_Address,2935 GIRARD AVE S,2935 GIRARD AVE S,2935 GIRARD AVE S,2935 GIRARD AVE S,2935 GIRARD AVE S
Addr Num,2935,2935,2935,2935,2935
Addr Num Suffix,NaN,NaN,NaN,NaN,NaN
Street Name,GIRARD,GIRARD,GIRARD,GIRARD,GIRARD
Street Type,AVE,AVE,AVE,AVE,AVE
Post Dir,S,S,S,S,S


In [16]:
#use for pulling data from geopandas web url 
def cleanInspections(df):
    df = df[df.InspectionType=="Routine"]
    df = df[df.FacilityCategory.isin(["RESTAURANT","GROCERY"])]
    df = df[df.YearOfInspection.isin([2017,2018,2019])]
    #df = df[df.ViolationPriority=="Priority1"]
    df = df.drop(labels=['OBJECTID',  
        'FacilityCategory','RiskLevel', 'FullAddress', 'AddrNum', 'AddrNumSuffix',
       'StreetName', 'StreetType', 'PostDir', 'Unit', 'City', 'State',
       'County', 'ZipCode','InspectionResult', 'FoodCodeText',
                         'ViolationStatus', 'ViolationPoints','FoodCodeItem',
       'InspectorComments', 'APN', 'Ward', 
                         'X_WebMercator', 'Y_WebMercator'
       ],axis=1)
    return(df)     

In [10]:
df.columns

Index(['LIC#RB', 'LIC_Cat', 'Business_Name', 'Risk Lvl', 'Full_Address',
       'Addr Num', 'Addr Num Suffix', 'Street Name', 'Street Type', 'Post Dir',
       'Unit', 'City', 'State', 'County', 'Zip Code', 'Inspection Type',
       'Inspection Result', 'Completed Date', 'Inspector Name', 'Src Insp Key',
       'Food Code Text', 'Comments', 'Priority', 'Standard Group',
       'Year Completed Date', 'Violation Status', 'ViolationPoints',
       'Inspection Score', 'Violation ID Number', 'APN', 'Ward',
       'Neighborhood', 'Latitude', 'Longitude', 'X WebMercator',
       'Y WebMercator'],
      dtype='object')

In [22]:
sorted(df.columns)

['addr_num',
 'addr_num_suffix',
 'apn',
 'business_name',
 'city',
 'comments',
 'completed_date',
 'county',
 'food_code_text',
 'full_address',
 'inspection_result',
 'inspection_score',
 'inspection_type',
 'inspector_name',
 'latitude',
 'lic#rb',
 'lic_cat',
 'longitude',
 'neighborhood',
 'post_dir',
 'priority',
 'risk_lvl',
 'src_insp_key',
 'standard_group',
 'state',
 'street_name',
 'street_type',
 'unit',
 'violation_id_number',
 'violation_status',
 'violationpoints',
 'ward',
 'x_webmercator',
 'y_webmercator',
 'year_completed_date',
 'zip_code']

In [36]:
df.street_name.value_counts()

Observed             176332
Corrected on-site     23111
Name: street_name, dtype: int64

In [41]:
df['lic#rb'].value_counts()

LIC44358     391
LIC53789     336
LIC41848     330
LIC47646     313
LIC49683     304
            ... 
LIC52998       1
LIC49980       1
LIC30785       1
LIC33347       1
LIC379448      1
Name: lic#rb, Length: 8050, dtype: int64

In [103]:
df=pd.read_excel('/Users/Simon/Documents/UW Data Science/DS785 Capstone/Minneapolis Open Data/Minneapolis_Inspections_Full.xlsx')

In [104]:
# use for cleaning data from excel
def cleanInspections(df):
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.lower()
    #df.columns = sorted(df.columns)
    df = df[df.inspection_type=="Routine"]
    df = df[df.lic_cat.isin(["RESTAURANT","GROCERY"])]
    #df = df[df.YearOfInspection.isin([2017,2018,2019])]
    #df = df[df.ViolationPriority=="Priority1"]
    df = df.drop(labels=[
        'addr_num',
        'addr_num_suffix',
        'apn',
        'city',
        'comments',
        'county',
        'food_code_text',
        'full_address',
        'inspection_result',
        'post_dir',
        'standard_group',
        'state',
        'street_name',
        'street_type',
        'unit',
        'violation_status',
        'violationpoints',
        'ward',
        'x_webmercator',
        'y_webmercator',
        'zip_code'
       ],axis=1)
    df=df.rename(columns={
        'lic#rb':'health_license_id',
        'lic_cat':'facility_category',
        'completed_date':'date_of_inspection',
        'src_insp_key':'inspection_id_number',
        'year_completed_date':'year_of_inspection'})
    df['priority_1_ind']=df.priority=='Priority1'
    df['priority_2_ind']=df.priority=='Priority2'
    df['priority_3_ind']=df.priority=='Priority3'
    return(df)

In [105]:
dfCleanFood=cleanInspections(df)

In [106]:
dfCleanFood.columns

Index(['health_license_id', 'facility_category', 'business_name', 'risk_lvl',
       'inspection_type', 'date_of_inspection', 'inspector_name',
       'inspection_id_number', 'priority', 'year_of_inspection',
       'inspection_score', 'violation_id_number', 'neighborhood', 'latitude',
       'longitude', 'priority_1_ind', 'priority_2_ind', 'priority_3_ind'],
      dtype='object')

In [107]:
dfCleanFood.head()

,health_license_id,facility_category,business_name,risk_lvl,inspection_type,date_of_inspection,inspector_name,inspection_id_number,priority,year_of_inspection,inspection_score,violation_id_number,neighborhood,latitude,longitude,priority_1_ind,priority_2_ind,priority_3_ind
1,LIC107101,RESTAURANT,SOTAROL UPTOWN,1,Routine,2017-07-15 15:30:00.000,Mohamed Yusuf,194952,Priority2,2017,94,625814.0,Lowry Hill East,44.94885,-93.29663,False,True,False
2,LIC107101,RESTAURANT,SOTAROL UPTOWN,1,Routine,2017-07-15 15:30:00.000,Mohamed Yusuf,194952,Priority1,2017,94,625816.0,Lowry Hill East,44.94885,-93.29663,True,False,False
3,LIC107101,RESTAURANT,SOTAROL UPTOWN,1,Routine,2018-07-18 13:49:59.999,Mohamed Yusuf,204578,Priority2,2018,92,625815.0,Lowry Hill East,44.94885,-93.29663,False,True,False
4,LIC107101,RESTAURANT,SOTAROL UPTOWN,1,Routine,2018-07-18 13:49:59.999,Mohamed Yusuf,204578,Priority3,2018,92,681981.0,Lowry Hill East,44.94885,-93.29663,False,False,True
5,LIC107101,RESTAURANT,SOTAROL UPTOWN,1,Routine,2018-07-18 13:49:59.999,Mohamed Yusuf,204578,Priority2,2018,92,681984.0,Lowry Hill East,44.94885,-93.29663,False,True,False


In [139]:
df=dfCleanFood

In [43]:
# df['priority_1_ind']=df.ViolationPriority=='Priority1'
# df['priority_2_ind']=df.ViolationPriority=='Priority2'
# df['priority_3_ind']=df.ViolationPriority=='Priority3'

Create a data frame that is grouped to the inspection level

In [140]:
# dfInspections=df.groupby(['BusinessName', 'DateOfInspection', 'HealthFacilityIDNumber',
#        'InspectionIDNumber', 'InspectionScore', 'InspectionType', 'Latitude',
#        'Longitude', 'Neighborhood',
#        'YearOfInspection']).aggregate({'priority_1_ind':'sum','priority_2_ind':'sum','priority_3_ind':'sum'}).reset_index()

dfInspections=df.groupby(['business_name', 'date_of_inspection', 'health_license_id',
       'inspection_id_number', 'inspection_score', 'inspection_type', 'latitude',
       'longitude', 'neighborhood', 'inspector_name','facility_category','risk_lvl',
       'year_of_inspection']).aggregate({'priority_1_ind':'sum','priority_2_ind':'sum','priority_3_ind':'sum'}).reset_index()

In [141]:
dfInspections

,business_name,date_of_inspection,health_license_id,inspection_id_number,inspection_score,inspection_type,latitude,longitude,neighborhood,inspector_name,facility_category,risk_lvl,year_of_inspection,priority_1_ind,priority_2_ind,priority_3_ind
0,1010 WASHINGTON WINE & SPIRITS,2015-02-27 10:10:00.000,LIC42954,44311,96,Routine,44.97659,-93.25502,Downtown East,Jim S Donovan,GROCERY,3,2015,1.0,0.0,1.0
1,1010 WASHINGTON WINE & SPIRITS,2017-01-27 12:19:59.999,LIC42954,44314,96,Routine,44.97659,-93.25502,Downtown East,Alexander S Kim,GROCERY,3,2017,1.0,0.0,1.0
2,1010 WASHINGTON WINE & SPIRITS,2019-02-06 13:20:00.000,LIC42954,189485,96,Routine,44.97659,-93.25502,Downtown East,Bob V Becker,GROCERY,3,2019,0.0,2.0,0.0
3,112 EATERY,2008-09-09 14:09:59.999,LIC42959,44316,82,Routine,44.98266,-93.27156,Downtown West,Ellen Hoyt,RESTAURANT,1,2008,3.0,4.0,3.0
4,112 EATERY,2009-11-07 15:45:00.000,LIC42959,44315,92,Routine,44.98266,-93.27156,Downtown West,Mohamed Yusuf,RESTAURANT,1,2009,1.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21537,ZUMBRO CAFE,2016-11-03 11:29:59.999,LIC30942,24280,92,Routine,44.92447,-93.31501,Linden Hills,Graham Miller,RESTAURANT,1,2016,2.0,0.0,10.0
21538,ZUMBRO CAFE,2016-11-03 11:29:59.999,LIC30944,24301,100,Routine,44.92447,-93.31501,Linden Hills,Graham Miller,RESTAURANT,1,2016,0.0,0.0,0.0
21539,ZUMBRO CAFE,2017-10-25 11:40:00.000,LIC30942,24300,80,Routine,44.92447,-93.31501,Linden Hills,Eduardo Mardones,RESTAURANT,1,2017,3.0,4.0,18.0
21540,ZUMBRO CAFE,2018-12-07 12:19:59.999,LIC30942,236468,96,Routine,44.92447,-93.31501,Linden Hills,SEID SHEKA,RESTAURANT,1,2018,0.0,2.0,4.0


In [142]:
# dfHealth=dfInspections[['HealthFacilityIDNumber','BusinessName']].groupby(['HealthFacilityIDNumber','BusinessName']).count().reset_index()
dfHealth=dfInspections[['health_license_id','business_name']].groupby(['health_license_id','business_name']).count().reset_index()

In [143]:
dfHealth.health_license_id.nunique()

5012

Each health facilty id number is assigned to a single business, even if two businesses are at the same location

In [144]:
dfHealth

,health_license_id,business_name
0,LIC107101,SOTAROL UPTOWN
1,LIC107124,HASKELL'S WINE AND CHEESE BAR
2,LIC107327,CRISP & GREEN
3,LIC107536,TIM HORTONS CAFE AND BAKE SHOP
4,LIC107553,YOGA CENTER OF MINNEAPOLIS
...,...,...
5007,LIC53824,WHOLE SUM KITCHEN
5008,LIC53841,WIILO FOOD DISTRIBUTOR
5009,LIC53856,WILDE ROAST CAFE
5010,LIC53860,WILDE ROAST CAFE


In [ ]:
# df.groupby(['InspectionIDNumber','ViolationPriority']).aggregate({'ViolationIDNumber':'count'})

## Create feature for time since last inspection

In [152]:
# insTimes=df.loc[:,['HealthFacilityIDNumber','DateOfInspection','InspectionIDNumber']]
# insTimes.DateOfInspection=pd.to_datetime(insTimes.DateOfInspection)
# insTimes=insTimes.groupby(['HealthFacilityIDNumber','DateOfInspection','InspectionIDNumber']).count()
# insTimes=insTimes.reset_index()

insTimes=df.loc[:,['health_license_id','date_of_inspection','inspection_id_number']]
insTimes.date_of_inspection=pd.to_datetime(insTimes.date_of_inspection)
insTimes=insTimes.groupby(['health_license_id','date_of_inspection','inspection_id_number']).count()
insTimes=insTimes.reset_index()

In [153]:
insTimes['years_since_last_inspect']=insTimes.sort_values(['health_license_id','date_of_inspection']).groupby('health_license_id')['date_of_inspection'].diff()
insTimes['years_since_last_inspect']=insTimes['years_since_last_inspect'].dt.days/365

In [154]:
insTimes[insTimes['years_since_last_inspect']>2].count()

health_license_id           401
date_of_inspection          401
inspection_id_number        401
years_since_last_inspect    401
dtype: int64

In [155]:
insTimes['first_inspection_ind']=insTimes.years_since_last_inspect.isna()

In [156]:
#Set missing time differences (first inspection) to 2 years
insTimes.loc[pd.isnull(insTimes.years_since_last_inspect),'years_since_last_inspect']=2
#Set time differences greater than 2 years to 2 years
insTimes.loc[insTimes['years_since_last_inspect']>2,'years_since_last_inspect']=2

In [157]:
insTimes.groupby('first_inspection_ind').aggregate({'years_since_last_inspect':'describe'})

years_since_last_inspect                          \
                                        count      mean      std  min   
first_inspection_ind                                                    
False                                 16781.0  1.131913  0.38572  0.0   
True                                   5099.0  2.000000  0.00000  2.0   

                                                        
                           25%      50%       75%  max  
first_inspection_ind                                    
False                 0.956164  0.99726  1.358904  2.0  
True                  2.000000  2.00000  2.000000  2.0

In [158]:
insTimes=insTimes[['inspection_id_number','years_since_last_inspect','first_inspection_ind']]

In [159]:
dfInspections=dfInspections.merge(insTimes)

In [160]:
dfInspections

,business_name,date_of_inspection,health_license_id,inspection_id_number,inspection_score,inspection_type,latitude,longitude,neighborhood,inspector_name,facility_category,risk_lvl,year_of_inspection,priority_1_ind,priority_2_ind,priority_3_ind,years_since_last_inspect,first_inspection_ind
0,1010 WASHINGTON WINE & SPIRITS,2015-02-27 10:10:00.000,LIC42954,44311,96,Routine,44.97659,-93.25502,Downtown East,Jim S Donovan,GROCERY,3,2015,1.0,0.0,1.0,2.000000,True
1,1010 WASHINGTON WINE & SPIRITS,2017-01-27 12:19:59.999,LIC42954,44314,96,Routine,44.97659,-93.25502,Downtown East,Alexander S Kim,GROCERY,3,2017,1.0,0.0,1.0,1.917808,False
2,1010 WASHINGTON WINE & SPIRITS,2019-02-06 13:20:00.000,LIC42954,189485,96,Routine,44.97659,-93.25502,Downtown East,Bob V Becker,GROCERY,3,2019,0.0,2.0,0.0,2.000000,False
3,112 EATERY,2008-09-09 14:09:59.999,LIC42959,44316,82,Routine,44.98266,-93.27156,Downtown West,Ellen Hoyt,RESTAURANT,1,2008,3.0,4.0,3.0,2.000000,True
4,112 EATERY,2009-11-07 15:45:00.000,LIC42959,44315,92,Routine,44.98266,-93.27156,Downtown West,Mohamed Yusuf,RESTAURANT,1,2009,1.0,2.0,3.0,1.161644,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21537,ZUMBRO CAFE,2016-11-03 11:29:59.999,LIC30942,24280,92,Routine,44.92447,-93.31501,Linden Hills,Graham Miller,RESTAURANT,1,2016,2.0,0.0,10.0,0.980822,False
21538,ZUMBRO CAFE,2016-11-03 11:29:59.999,LIC30944,24301,100,Routine,44.92447,-93.31501,Linden Hills,Graham Miller,RESTAURANT,1,2016,0.0,0.0,0.0,0.980822,False
21539,ZUMBRO CAFE,2017-10-25 11:40:00.000,LIC30942,24300,80,Routine,44.92447,-93.31501,Linden Hills,Eduardo Mardones,RESTAURANT,1,2017,3.0,4.0,18.0,0.975342,False
21540,ZUMBRO CAFE,2018-12-07 12:19:59.999,LIC30942,236468,96,Routine,44.92447,-93.31501,Linden Hills,SEID SHEKA,RESTAURANT,1,2018,0.0,2.0,4.0,1.117808,False


In [161]:
# dfInspections['DateOfInspection']=pd.to_datetime(dfInspections.DateOfInspection)
dfInspections['date_of_inspection']=pd.to_datetime(dfInspections.date_of_inspection)

In [162]:
dfInspections=dfInspections.sort_values(['date_of_inspection','health_license_id'])

In [163]:
dfInspections['prior_priority_1'] = dfInspections.groupby(['health_license_id'])['priority_1_ind'].shift()

In [164]:
dfInspections['prior_priority_2'] = dfInspections.groupby(['health_license_id'])['priority_2_ind'].shift()

In [165]:
dfInspections['prior_priority_3'] = dfInspections.groupby(['health_license_id'])['priority_3_ind'].shift()

In [166]:
dfInspections[['prior_priority_1','prior_priority_2','prior_priority_3']]=dfInspections[['prior_priority_1','prior_priority_2','prior_priority_3']].fillna(0)

In [167]:
dfInspections[dfInspections.health_license_id=='LIC47206']

,business_name,date_of_inspection,health_license_id,inspection_id_number,inspection_score,inspection_type,latitude,longitude,neighborhood,inspector_name,...,risk_lvl,year_of_inspection,priority_1_ind,priority_2_ind,priority_3_ind,years_since_last_inspect,first_inspection_ind,prior_priority_1,prior_priority_2,prior_priority_3
410,AGRA CULTURE,2007-12-27 15:30:00.000,LIC47206,51858,98,Routine,44.95821,-93.27404,Whittier,Mohamed Yusuf,...,1,2007,0.0,1.0,3.0,2.000000,True,0.0,0.0,0.0
411,AGRA CULTURE,2008-09-03 10:49:59.999,LIC47206,51857,100,Routine,44.95821,-93.27404,Whittier,Bob V Becker,...,1,2008,0.0,0.0,7.0,0.684932,False,0.0,1.0,3.0
412,AGRA CULTURE,2010-01-12 11:05:00.000,LIC47206,51852,100,Routine,44.95821,-93.27404,Whittier,Bob V Becker,...,1,2010,0.0,0.0,3.0,1.358904,False,0.0,0.0,7.0
413,AGRA CULTURE,2011-02-03 13:30:00.000,LIC47206,51849,90,Routine,44.95821,-93.27404,Whittier,Kathy L Louden,...,1,2011,2.0,1.0,2.0,1.060274,False,0.0,0.0,3.0
414,AGRA CULTURE,2012-01-06 13:40:00.000,LIC47206,51847,94,Routine,44.95821,-93.27404,Whittier,Kathy L Louden,...,1,2012,2.0,1.0,6.0,0.923288,False,2.0,1.0,2.0
415,AGRA CULTURE,2013-01-02 12:39:59.999,LIC47206,51845,100,Routine,44.95821,-93.27404,Whittier,Kathy L Louden,...,1,2013,0.0,0.0,2.0,0.989041,False,2.0,1.0,6.0
416,AGRA CULTURE,2014-03-13 12:30:00.000,LIC47206,51841,94,Routine,44.95821,-93.27404,Whittier,Anne Stahn,...,1,2014,1.0,2.0,5.0,1.189041,False,0.0,0.0,2.0
419,AGRA CULTURE,2015-02-06 14:00:00.000,LIC47206,51839,96,Routine,44.95821,-93.27404,Whittier,Anne Stahn,...,1,2015,4.0,2.0,3.0,0.904110,False,1.0,2.0,5.0
425,AGRA CULTURE,2016-01-21 14:50:00.000,LIC47206,51837,92,Routine,44.95821,-93.27404,Whittier,Kevin A Keopraseuth,...,1,2016,0.0,4.0,4.0,0.956164,False,4.0,2.0,3.0
428,AGRA CULTURE,2017-01-19 15:00:00.000,LIC47206,51859,96,Routine,44.95821,-93.27404,Whittier,Kevin A Keopraseuth,...,1,2017,1.0,0.0,1.0,0.997260,False,0.0,4.0,4.0


In [168]:
dfInspections['priority_1_ind']=dfInspections['priority_1_ind']>0
dfInspections['priority_2_ind']=dfInspections['priority_2_ind']>0
dfInspections['priority_3_ind']=dfInspections['priority_3_ind']>0

In [169]:
dfInspections=gpd.GeoDataFrame(dfInspections, geometry=gpd.points_from_xy(dfInspections.longitude, dfInspections.latitude), crs={'init' :'epsg:4326'})

In [192]:
earliest_inspection=dfInspections.groupby('health_license_id').aggregate(earliest_inspection=('date_of_inspection','min'))

In [196]:
dfInspections=dfInspections.join(earliest_inspection,on='health_license_id')

In [197]:
dfInspections.inspection_id_number.nunique()

21542

In [199]:
dfInspections['years_since_first_inspection']=dfInspections['date_of_inspection']- dfInspections['earliest_inspection']

In [201]:
dfInspections['years_since_first_inspection']=dfInspections['years_since_first_inspection'].dt.days/365

In [203]:
dfInspections['years_since_first_inspection'].describe()

count    21542.000000
mean         3.175028
std          3.044964
min          0.000000
25%          0.691096
50%          2.317808
75%          5.054795
max         12.093151
Name: years_since_first_inspection, dtype: float64

In [176]:
len(dfInspections)

21542

In [173]:
dfInspections.nunique()

business_name                3832
date_of_inspection          18560
health_license_id            5012
inspection_id_number        21542
inspection_score               48
inspection_type                 1
latitude                     1348
longitude                    1344
neighborhood                   86
inspector_name                 64
facility_category               2
risk_lvl                        4
year_of_inspection             14
priority_1_ind                  2
priority_2_ind                  2
priority_3_ind                  2
years_since_last_inspect      711
first_inspection_ind            2
prior_priority_1               13
prior_priority_2               13
prior_priority_3               31
geometry                     1628
dtype: int64

In [204]:
dfInspections.to_pickle('dfInspections.pickle')